<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/PVTtableGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Generation of property tables for flow simulators
#@markdown The fundamental components for fluid handling are available including, predictive characterization, flexible lumping, mixing and recombining gas and oil compositions. Other functions available with this package include plotting phase envelopes and calculating equilibrium gas-liquid-aqueous phase properties using PT (pressure, temperature) flash operations,
%%capture
!pip install neqsim==2.5.35
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
from neqsim.thermo import fluid
%matplotlib inline

# Generation of property tables for OLGA/Ledaflow #

The following example demonstrates how to use NeqSim for generation of property tables for OLGA/Ledaflow. The property file will be written to the users google disk.
The calculations are done in two steps:
1. Define a fluid in neqsim
2. Run property generation routine in a temperature and pressure range.



Theory:
See work of [Bjørtuft](https://ntnuopen.ntnu.no/ntnu-xmlui/handle/11250/235677).


## 1. Define the fluid in neqsim
We start by defining a fluid in neqsim. Components are added and mole fraction of the components are specified.


In [ ]:
fluid1 = fluid('srk')
fluid1.addComponent("nitrogen", 1.0)
fluid1.addComponent("CO2", 2.0)
fluid1.addComponent("methane", 79.2)
fluid1.addComponent("ethane", 7.3)
fluid1.addComponent("propane", 2.3)
fluid1.addComponent("i-butane", 1.0)
fluid1.addComponent("n-butane", 0.7)
fluid1.addComponent("i-pentane", 0.04)
fluid1.addComponent("n-pentane", 0.02)
fluid1.addComponent("n-hexane", 0.003)
fluid1.addComponent("water", 1.3)
fluid1.setMixingRule('classic')
fluid1.setMultiPhaseCheck(True)

# 2. Run property generation routine

The property genreation routine is run in a temperature and pressure range. The calculations are done by giving a input fluid composition and providing a temperature and pressure range. The number of temperature and pressure steps in the table needs to be provided.

The results are saved in a file in the Colab drive (accessible via left menu).

In [ ]:
fileName = "MytestPVT.tab"

lowTemperature= 273.15 # K
highTemperature = 373.0 # k
Tsteps = 20
lowPressure = 1.0 # bara
highPressure = 100.0 # bara
Psteps = 20

PVTpropTable(fluid1, fileName, lowTemperature, highTemperature, Tsteps, lowPressure, highPressure, Psteps)


#3. Generate PVT file for gas-condensate including water




In [ ]:
from neqsim.thermo import fluid, fluid_df, printFrame
import pandas as pd

gascondensate = {'ComponentName':   ['water', 'nitrogen','CO2', 'methane', 'ethane', 'propane','i-butane','n-butane','i-pentane','n-pentane',"C6", "C7", "C8", "C9", "C10"],
        'MolarComposition[-]':  [4.0, 0.972, 0.632, 91.111, 2.553, 0.104, 0.121, 0.021, 0.066, 0.02,0.058, 0.107, 0.073, 0.044, 0.118],
        'MolarMass[kg/mol]': [None, None, None, None, None, None, None, None, None, None, 0.08618, 0.096, 0.107, 0.121, 0.202],
        'RelativeDensity[-]': [None, None, None, None, None, None, None, None, None, None, 664.0e-3, 738.0e-3, 765.0e-3, 781.0e-3, 813.30e-3]}

compositionDataFrame = pd.DataFrame(gascondensate)
print(compositionDataFrame.head(20).to_string())

gascondensatedf = pd.DataFrame(gascondensate)
print(gascondensatedf.head(30).to_string())
gascondensateFluid = fluid_df(gascondensatedf, lastIsPlusFraction=True, numberOfLumpedComponents=12)
gascondensateFluid.setMixingRule('classic')
gascondensateFluid.setMultiPhaseCheck(True)

#Run a flash and check composition
TPflash(gascondensateFluid, temperature=20.0, tUnit='C', pressure=11.01325, pUnit='bara' )
printFrame(gascondensateFluid)

fileName = "MytestPVT_fullfluid.tab"

lowTemperature= 273.15 # K
highTemperature = 373.0 # k
Tsteps = 20
lowPressure = 1.0 # bara
highPressure = 100.0 # bara
Psteps = 20

PVTpropTable(fluid1, fileName, lowTemperature, highTemperature, Tsteps, lowPressure, highPressure, Psteps)

   ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0          water                4.000                NaN                 NaN
1       nitrogen                0.972                NaN                 NaN
2            CO2                0.632                NaN                 NaN
3        methane               91.111                NaN                 NaN
4         ethane                2.553                NaN                 NaN
5        propane                0.104                NaN                 NaN
6       i-butane                0.121                NaN                 NaN
7       n-butane                0.021                NaN                 NaN
8      i-pentane                0.066                NaN                 NaN
9      n-pentane                0.020                NaN                 NaN
10            C6                0.058            0.08618              0.6640
11            C7                0.107            0.09600              0.7380

#Print PVT file

In [ ]:
file1 = open('MytestPVT_fullfluid.tab', "r")
FileContent = file1.read()
print(FileContent)

'WATER-OPTION ENTROPY NONEQ '
20   20    0.0
100000.0    621052.6315789473    1142105.2631578946    1663157.894736842    2184210.526315789    
2705263.1578947366    3226315.789473684    3747368.4210526315    4268421.052631578    4789473.684210526    
5310526.315789473    5831578.947368421    6352631.578947368    6873684.2105263155    7394736.842105263    
7915789.473684209    8436842.105263157    8957894.736842105    9478947.368421052    1.0E7    
0.0    5.255263157894717    10.510526315789491    15.765789473684208    21.021052631578982    
26.2763157894737    31.531578947368416    36.78684210526319    42.04210526315791    47.297368421052624    
52.5526315789474    57.807894736842115    63.06315789473683    68.3184210526316    73.57368421052632    
78.82894736842104    84.08421052631581    89.33947368421059    94.5947368421053    99.85000000000002    
0.0    0.0    0.0    0.0    0.0    
0.0    0.0    0.0    0.0    0.0    
0.0    0.0    0.0    0.0    0.0    
0.0    0.0    0.0    0.0    